In [ ]:
import os

os.chdir('/opt/project')
print(os.getcwd())

In [20]:
import SimpleITK as sitk
from pydicom import dcmread
from pydicom.fileset import FileSet

def inspect_dicom(folder_path):
#     reader = sitk.ImageSeriesReader()
#     dicom_names = reader.GetGDCMSeriesFileNames(folder_path)
#     reader.SetFileNames(dicom_names)
#     image = reader.Execute()
    
#     reader.LoadPrivateTagsOn()
#     reader.ReadImageInformation()
    
#     for k in reader.GetMetaDataKeys():
#         v = reader.GetMetaData(k)
#         print(f'({k}) = = "{v}"')

#     print(f"Image Size: {reader.GetSize()}")
#     print(f"Image PixelType: {sitk.GetPixelIDValueAsString(reader.GetPixelID())}")
    dicom_dir = dcmread(folder_path)

    # Iterate over patient records
    for patient_record in dicom_dir.patient_records:
        patient_id = patient_record.PatientID
        print(f"Patient ID: {patient_id}")

        # Iterate over studies
        for study in patient_record.children:
            study_date = study.StudyDate
            study_description = study.StudyDescription
            print(f"\tStudy Date: {study_date}, Study Description: {study_description}")

            # Iterate over series
            for series in study.children:
#                 print(series)
                series_description = series.SeriesNumber
                print(f"\t\tSeries Number: {series_description}")
                
                # Iterate over instances
                for instance in series.children:
                    print('-'*100)
                    print(f"Instance Number {instance.InstanceNumber}")
                    print(f"Image Position (Patient): {instance.ImagePosition}")
                    print(f"Image Orientation(Patient): {instance.ImageOrientation}")
                    print(f"Image Orientation(Patient): {instance.ImageOrientation}")
                    
                    
                    
                    print()
#                     instance_uid = instance.SOPInstanceUID
#                     instance_number = instance.InstanceNumber
#                     print(f"\t\t\tInstance UID: {instance_uid}, Instance Number: {instance_number}")
                    print(instance)

    

inspect_dicom('data/sarcoma_dicom/SARC_00001/DICOMDIR')
    

Patient ID: 86.2138451
	Study Date: 20240201, Study Description: CRIC - MRI - SARCOMA MRI STUDY
		Series Number: 6
----------------------------------------------------------------------------------------------------
(0004, 1400) Offset of the Next Directory Record UL: 1514
(0004, 1410) Record In-use Flag                  US: 65535
(0004, 1420) Offset of Referenced Lower-Level Di UL: 0
(0004, 1430) Directory Record Type               CS: 'IMAGE'
(0004, 1500) Referenced File ID                  CS: ['DICOM', 'PA000002', 'ST000001', 'SE000001', 'IM000001']
(0004, 1510) Referenced SOP Class UID in File    UI: MR Image Storage
(0004, 1511) Referenced SOP Instance UID in File UI: 1.3.12.2.1107.5.2.63.213008.30000024013011190945100156305
(0004, 1512) Referenced Transfer Syntax UID in F UI: Explicit VR Little Endian
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['DERIVED', 'PRIMARY', 'DIFFUSION', 'ADC', 'NORM', 'DRS', 'DI

(0004, 1400) Offset of the Next Directory Record UL: 231056
(0004, 1410) Record In-use Flag                  US: 65535
(0004, 1420) Offset of Referenced Lower-Level Di UL: 0
(0004, 1430) Directory Record Type               CS: 'IMAGE'
(0004, 1500) Referenced File ID                  CS: ['DICOM', 'PA000003', 'ST000001', 'SE000005', 'IM000139']
(0004, 1510) Referenced SOP Class UID in File    UI: MR Image Storage
(0004, 1511) Referenced SOP Instance UID in File UI: 1.3.12.2.1107.5.2.63.213008.30000024013011190945100156199
(0004, 1512) Referenced Transfer Syntax UID in F UI: Explicit VR Little Endian
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'DIFFUSION', 'NONE', 'NORM', 'DRS', 'DIS2D', 'DFC', 'MIX', 'DRB', 'MFSPLIT']
(0008, 002a) Acquisition DateTime                DT: '20240201123315.567500'
(0020, 0013) Instance Number                     IS: '139'
(0020, 0032) Image Position (Patient) 

In [31]:
import importlib
importlib.reload(importlib.import_module("src.models.utils"))
importlib.reload(importlib.import_module("src.models.autoencoder"))
importlib.reload(importlib.import_module("src.models.autoencoder.fmcib_model"))
from src.models.autoencoder.fmcib_model import load_fmcib_model

model = load_fmcib_model(save_path='outputs/pretrained_models', input_channels=5)
model

Missing keys: [] and unexpected keys: []


FMCIBModel(
  (trunk): ResNet(
    (conv1): Conv3d(5, 128, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
    (bn1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): ResNetBottleneck(
        (conv1): Conv3d(128, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv3d(128, 512, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn3): BatchNorm3d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True

In [27]:
from src.dataset.custom_dicom2nifti import dicomdir2nii, get_dicom_meta, get_dicom_meta, get_dicomdir_meta

dataset_meta = dicomdir2nii('./data/sarcoma_dicom', './data/sarcoma_nifti', series_description_filter=['T2.*AX','DIFF.*ADC'], codependent=True, codependent_tolerance=0.5)
dataset_meta.to_csv('data/sarcoma_meta.csv')

odict_keys(['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.num_batches_tracked', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.num_batches_tracked', 'layer1.0.conv3.weight', 'layer1.0.bn3.weight', 'layer1.0.bn3.bias', 'layer1.0.bn3.running_mean', 'layer1.0.bn3.running_var', 'layer1.0.bn3.num_batches_tracked', 'layer1.0.downsample.0.weight', 'layer1.0.downsample.0.bias', 'layer1.0.downsample.1.weight', 'layer1.0.downsample.1.bias', 'layer1.0.downsample.1.running_mean', 'layer1.0.downsample.1.running_var', 'layer1.0.downsample.1.num_batches_tracked', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.num_batche

In [28]:
model.state_dict().keys()

odict_keys(['trunk.conv1.weight', 'trunk.bn1.weight', 'trunk.bn1.bias', 'trunk.bn1.running_mean', 'trunk.bn1.running_var', 'trunk.bn1.num_batches_tracked', 'trunk.layer1.0.conv1.weight', 'trunk.layer1.0.bn1.weight', 'trunk.layer1.0.bn1.bias', 'trunk.layer1.0.bn1.running_mean', 'trunk.layer1.0.bn1.running_var', 'trunk.layer1.0.bn1.num_batches_tracked', 'trunk.layer1.0.conv2.weight', 'trunk.layer1.0.bn2.weight', 'trunk.layer1.0.bn2.bias', 'trunk.layer1.0.bn2.running_mean', 'trunk.layer1.0.bn2.running_var', 'trunk.layer1.0.bn2.num_batches_tracked', 'trunk.layer1.0.conv3.weight', 'trunk.layer1.0.bn3.weight', 'trunk.layer1.0.bn3.bias', 'trunk.layer1.0.bn3.running_mean', 'trunk.layer1.0.bn3.running_var', 'trunk.layer1.0.bn3.num_batches_tracked', 'trunk.layer1.0.downsample.0.weight', 'trunk.layer1.0.downsample.0.bias', 'trunk.layer1.0.downsample.1.weight', 'trunk.layer1.0.downsample.1.bias', 'trunk.layer1.0.downsample.1.running_mean', 'trunk.layer1.0.downsample.1.running_var', 'trunk.layer1.0